In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Load Date into Pandas Data Frame

In [5]:
zip_file = "Home_value_by_zip.csv"
zip_df = pd.read_csv(zip_file, encoding = 'latin-1')
zip_df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2000-01,2000-02,2000-03,...,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,314300.0,323500.0,332200.0,...,1078900,1071800,1067600,1054400,1030700,1015300,1008600,988500,971100,966900
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,216500.0,220300.0,224000.0,...,348500,350400,352500,353100,352900,353100,352400,350400,347500,344900
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,523900.0,531400.0,538300.0,...,1410500,1391600,1384300,1377900,1370900,1370000,1361800,1354700,1365700,1385100
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,313500.0,315000.0,316700.0,...,338100,336800,336100,336200,335500,334100,333400,333600,333400,332300
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,262000.0,266300.0,270300.0,...,429500,429500,431300,432400,432500,432200,430600,428400,425800,423100


### Transform and Clean

In [6]:
zip_df = zip_df[['RegionName', 'City', 'State', 'CountyName', '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06']].copy()

In [7]:
zip_df.head()

,RegionName,City,State,CountyName,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06
0,10025,New York,NY,New York County,1030700,1015300,1008600,988500,971100,966900
1,60657,Chicago,IL,Cook County,352900,353100,352400,350400,347500,344900
2,10023,New York,NY,New York County,1370900,1370000,1361800,1354700,1365700,1385100
3,77494,Katy,TX,Harris County,335500,334100,333400,333600,333400,332300
4,60614,Chicago,IL,Cook County,432500,432200,430600,428400,425800,423100


In [8]:
zip_df.columns = ['ZipCode', 'City', 'State', 'County', '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06']
zip_df.head()

,ZipCode,City,State,County,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06
0,10025,New York,NY,New York County,1030700,1015300,1008600,988500,971100,966900
1,60657,Chicago,IL,Cook County,352900,353100,352400,350400,347500,344900
2,10023,New York,NY,New York County,1370900,1370000,1361800,1354700,1365700,1385100
3,77494,Katy,TX,Harris County,335500,334100,333400,333600,333400,332300
4,60614,Chicago,IL,Cook County,432500,432200,430600,428400,425800,423100


In [14]:
zip_df['Sum Home Value'] = zip_df['2019-01'] + zip_df['2019-01'] + zip_df['2019-02'] + zip_df['2019-03'] + zip_df['2019-04'] + zip_df['2019-05'] + zip_df['2019-06']
zip_df['Avg 2019 Home Value'] = round(zip_df['Sum Home Value'] / 6,2)
zip_df.head()

,ZipCode,City,State,County,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,Sum Home Value,Avg 2019 Home Value
0,10025,New York,NY,New York County,1030700,1015300,1008600,988500,971100,966900,7011800,1168633.33
1,60657,Chicago,IL,Cook County,352900,353100,352400,350400,347500,344900,2454100,409016.67
2,10023,New York,NY,New York County,1370900,1370000,1361800,1354700,1365700,1385100,9579100,1596516.67
3,77494,Katy,TX,Harris County,335500,334100,333400,333600,333400,332300,2337800,389633.33
4,60614,Chicago,IL,Cook County,432500,432200,430600,428400,425800,423100,3005100,500850.00


In [17]:
clean_zip_df = zip_df[['ZipCode', 'City', 'State', 'County', 'Avg 2019 Home Value']]
pd.options.display.float_format = '${:,.2f}'.format

In [18]:
clean_zip_df.head()

,ZipCode,City,State,County,Avg 2019 Home Value
0,10025,New York,NY,New York County,"$1,168,633.33"
1,60657,Chicago,IL,Cook County,"$409,016.67"
2,10023,New York,NY,New York County,"$1,596,516.67"
3,77494,Katy,TX,Harris County,"$389,633.33"
4,60614,Chicago,IL,Cook County,"$500,850.00"


### Create Database Connection

In [ ]:
rds_connection_string = f"postgres:PASSWORD@localhost:5432/campaign_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

### Load into Database

In [ ]:
clean_zip_df.to_sql(name='clean_zip_df', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from clean_zip_df', con=engine).head()